In [ ]:
import pandas as pd

df = pd.read_csv('bphn.csv')

In [ ]:
df.info()

In [ ]:
df.link

In [ ]:
df2 = df.copy

In [ ]:
df.drop_duplicates(subset=['link'])

In [ ]:
df.drop_duplicates()

In [ ]:
df_new= df[["link"]].copy()

In [ ]:
df_new.info()

In [ ]:
df_new

In [ ]:
df_new.dropna(inplace=True)

In [ ]:
len(df_new)

In [ ]:
df_new

In [ ]:
list(df_new)

In [ ]:
# download all files from a website sequentially
import requests
from os import makedirs
from os.path import basename
from os.path import join
from urllib.request import urlopen
from urllib.parse import urljoin
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed
from concurrent.futures import wait
from random import randint
from time import sleep
 
# load a file from a URL, returns content of downloaded file
def download_url(urlpath):
    # open a connection to the server
    with urlopen(urlpath) as connection:
        # read the contents of the url as bytes and return it
        return connection.read()

# save provided content to the local path
def save_file(path, data):
    # open the local file for writing
    with open(path, 'wb') as file:
        # write all provided data to the file
        file.write(data)

# download one file to a local directory
def download_url_to_file(url, link, path):
    sleep(randint(1,5))
    r= requests.get(link,allow_redirects=True)
    link = r.url
    print('trying '+link)

    if not exists(link):
        print('URL Not exist')
        return None
    # skip bad urls or bad filenames
    if link is None or link == '../':
        print('Link is None')
        return (link, None)
    # check for no file extension
    if not (link[-4] == '.' or link[-3] == '.' ):
        print('Link is not a file')
        return (link, None)
    # convert relative link to absolute link
    absurl = link #urljoin(url, link)
    # download the content of the file

    # get the filename
    filename = basename(absurl)
    # construct the output path
    outpath = join(path, filename)

    if(exists(outpath)):
        print('File already exist')
        return (link, None)
    data = download_url(absurl)
    
    # save to file
    save_file(outpath, data)
    # return results
    return (link, outpath)

def exists(path):
    r = requests.head(path)
    return r.status_code == requests.codes.ok

# download all files on the provided webpage to the provided path
def download_all_files(mydf, path):
    # create a local directory to save files
    makedirs(path, exist_ok=True)
    # parse html and retrieve all href urls listed
    links = mydf.link
    # report progress
    print(f'Found {len(links)} links in data')
    # create the pool of worker threads
    with ThreadPoolExecutor(max_workers=5) as exe:
        # dispatch all download tasks to worker threads

        futures = [exe.submit(download_url_to_file, 'https://jdihn.go.id/', link, path) for link in links]
        # report results as they become available
        for future in as_completed(futures):
            # retrieve result
            link, outpath = future.result()
            print(outpath)
            # check for a link that was skipped
            if outpath is None:
                print(f'>skipped {link}')
            else:
                print(f'Downloaded {link} to {outpath}')
        wait(futures)  
        print('All tasks are done!')      


In [ ]:
len(df_new)

In [ ]:
PATH = 'bphn'
# download all files on df link
download_all_files(df_new,PATH)